## Society of Minds

* An agent that uses an inner team of agents to generate responses
* Each time the agent's on_message() or on_message_stream() method is called it runs inner team of agents  

In [1]:
from autogen_agentchat.agents import AssistantAgent
import asyncio
import os
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core.models import ModelInfo
from dotenv import load_dotenv

In [2]:
model_info = ModelInfo(
    model="gemini-2.5-pro",
    is_chat_model=True,
    is_function_calling_model=True ,
    vision=False,
    function_calling=True,
    json_output=True,
    family='unknown',
    multiple_system_messages=True
      # Set to False if the model doesn't support function calling
)

In [3]:
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
model_client = OpenAIChatCompletionClient(model='gemini-2.5-pro',api_key=api_key, model_info=model_info)
my_assistant = AssistantAgent(name='Assistant',model_client=model_client)

c:\Users\mukun\anaconda3\envs\autogen\lib\site-packages\autogen_ext\models\openai\_openai_client.py:439: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [4]:

agent1 = AssistantAgent(
    name="loan_data_collector_agent",
    model_client=model_client,
    description="An agent to provide prospective loan application profile summary",
    system_message=(
        "You are a loan data collector. Given an applicant profile, summarize "
        "their income, debts, credit score, and requested loan amount clearly."
    )
)

agent2 = AssistantAgent(
    name = 'risk_analyzer_agent',
    model_client = model_client,
    description='An agent to analyse the risk profile of the provided applicant',
    system_message=""" You are a credit risk analyzer agent. Review the application profile summary
      provided by the loan_data_collector_agent and provide an assessment of the risk of default in under 30 words """
)

In [5]:
agent3 = AssistantAgent(
    name="loan_amount_recommender_agent",
    model_client=model_client,
    system_message=(
        """ You are a loan amount recommender. Based on the applicant profile and the 
         assessed risk, provide a suitable loan amount for the application. If you think that
          the loan applicant is not eligbile for any loan reply with 0. """
    )
)

human_agent = UserProxyAgent(
    name = 'human_supervisor_agent',
    description="""You are the final loan officer. Review all information provided.
        - Type APPROVE to approve the loan.
        - Type REJECT to reject.
        - Provide feedback to request changes or more analysis."""
    
)

In [6]:
termination_condition = TextMentionTermination("APPROVE")
inner_team = RoundRobinGroupChat(
    [agent1, agent2, agent3, human_agent],
    termination_condition=termination_condition,
    max_turns=8
)

In [7]:
society_of_mind_agent = SocietyOfMindAgent(
    name="society_of_mind",
    model_client=model_client,
    team=inner_team,
    #always_use_team = True,
    response_prompt=(
        "Retain all relevant reasoning and intermediate outputs from loan_collector_agent, "
        "risk_analyzer_agent, loan_amount_recommender_agent, and human_supervisor_agentr. Output exactly the "
        "final approved or rejected decision along with key facts and reasoning."
    )
)

In [8]:
agent4 = AssistantAgent(
    name="loan_offer_generator_agent",
    model_client=model_client,
    description='An agent to provide the final response to the applicant',
    system_message="""Given the final decision summary, draft a short, formal message for the applicant.
    Keep it to under 30 words."""
)

In [9]:
outer_team = RoundRobinGroupChat([society_of_mind_agent, agent4], max_turns=2)

In [10]:
async def main():
    task = """Applicant info: Income=20L/year, debts=3L, credit score=720,
                requested=3L for business expansion. Data is unverified — please process
                All amounts are in Indian rupees"""
    stream = outer_team.run_stream(task=task)
    await Console(stream)

In [11]:
await main()

---------- TextMessage (user) ----------
Applicant info: Income=20L/year, debts=3L, credit score=720,
                requested=3L for business expansion. Data is unverified — please process
                All amounts are in Indian rupees


---------- TextMessage (loan_data_collector_agent) ----------
Understood. Here is the summary of the applicant's profile:

**Applicant Financial Profile Summary**

*   **Annual Income:** ₹20 Lakh
*   **Existing Debts:** ₹3 Lakh
*   **Credit Score:** 720
*   **Requested Loan Amount:** ₹3 Lakh
*   **Loan Purpose:** Business Expansion

**Status:** All provided data is currently unverified.
---------- TextMessage (risk_analyzer_agent) ----------
While the profile appears strong, the unverified data represents a high default risk. Mandatory verification of all financial details is required before proceeding.
---------- TextMessage (loan_amount_recommender_agent) ----------
Based on the information provided, here is the loan recommendation.

### **Loan Recommendation**

**Recommended Loan Amount: ₹0**

### **Risk Assessment and Rationale**

The primary and overriding risk factor is that **all applicant data is unverified**.

While the applicant's stated financial profile appears strong on th